#### Testing out Gibbs zig-zag code:

In [1]:
using Distributions, TimeIt, ProgressMeter, PyPlot, JLD
include("zz_structures.jl")
include("mbsampler.jl")

get_clustering (generic function with 1 method)

In [2]:
X = load("GZZ_data.jld", "X")
y = load("GZZ_data.jld", "y")
ξ_true = load("GZZ_data.jld", "xi_true");

d, Nobs = size(X)
mb_size = Int(round(Nobs/10))
σ02 = 1
prior = HS_prior(d, σ02) 

HS_prior(50, 1.0, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 1.0, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 1.0)

In [3]:
ξ_true[ξ_true .!= 0]

7-element Array{Float64,1}:
 -0.731787
 -6.2554  
  9.27311 
  6.42829 
  7.38964 
 -7.66661 
 -8.43724 

In [4]:
my_ll = ll_logistic(X,y);
my_model = model(my_ll, prior)
root = find_root(my_model, rand(d));

In [21]:
# Sub-Sampling without control variate and with weights
ϵ = 1e-2
weights = abs.(X) + ϵ
weights ./= sum(weights,2)
gs = [wumbsampler(Nobs, mb_size, weights[i,:]) for i in 1:d]
gs_list = mbsampler_list(d,gs);

In [22]:
A = eye(d)

opf = projopf(A, 1000, hyperparam_size(prior))
opt = maxa_opt(10^7)
outp = outputscheduler(opf,opt)
bb = linear_bound(my_model.ll, my_model.pr, gs_list, zz_state(opf))
update_bound(bb, my_ll, prior, gs_list, zz_state(opf));

In [23]:
print("Initial speed = ", opf.alpha_skeleton[:,opf.tcounter+1])

Initial speed = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [24]:
adapt_speed = false
L = 1
my_zz_sampler = zz_sampler(0, gs_list, bb, L, adapt_speed)
hyper_sampler = block_gibbs_sampler(1.)
blocksampler = Array{msampler}(2)
blocksampler[1] = my_zz_sampler
blocksampler[2] = hyper_sampler;

In [ ]:
ZZ_block_sample(my_model::model, outp::outputscheduler, blocksampler::Array{msampler});

10% attempts in 3.42 mins 
20% attempts in 7.54 mins 
30% attempts in 12.35 mins 
40% attempts in 17.33 mins 
50% attempts in 22.62 mins 


In [ ]:
discard = 0
n_samples = 10^4
xi_samples = extract_samples(outp.opf.xi_skeleton[:,discard+1:end], 
                             outp.opf.bt_skeleton[:,discard+1:end], 
                             outp.opf.bt_skeleton[end]/n_samples,
                             "linear")[:,1:end-1];

In [ ]:
maxlag = 100
acfs = zeros(d, maxlag)
for i in 1:d 
    acfs[i,:] = acf(xi_samples[i,:], maxlag)
end

In [ ]:
for i in 1:d 
    plot(acfs[i,:])
end
grid(true)

In [ ]:
cover = zeros(d)
for i in 1:d 
    ci = percentile(xi_samples[i,:], [5, 95])
    cover[i] = (ci[1]<ξ_true[i])&(ξ_true[i]<ci[2])
end

In [ ]:
100*mean(cover)